In [2]:
import pandas as pd
import sqlite3, requests, json, time, hmac
from ta.trend import SMAIndicator, EMAIndicator

In [3]:
ENDPOINT = "https://api.binance.com/"

Check for connection to the API

In [4]:

r = requests.get(ENDPOINT + "api/v3/exchangeInfo")
r.status_code

200

# GET crypto datas

Get all avaible cryptocurrencies.

In [5]:
def avaibleCrypto():
  r = requests.get(ENDPOINT + "api/v3/exchangeInfo")
  assets = json.loads(r.text)
  ListSymbols = assets["symbols"]
  baseAsset = []
  for elem in ListSymbols:
    baseAsset.append(elem['baseAsset'])

  return baseAsset

In [6]:
avaibleCrypto()

['ETH',
 'LTC',
 'BNB',
 'NEO',
 'QTUM',
 'EOS',
 'SNT',
 'BNT',
 'BCC',
 'GAS',
 'BNB',
 'BTC',
 'ETH',
 'HSR',
 'OAX',
 'DNT',
 'MCO',
 'ICN',
 'MCO',
 'WTC',
 'WTC',
 'LRC',
 'LRC',
 'QTUM',
 'YOYO',
 'OMG',
 'OMG',
 'ZRX',
 'ZRX',
 'STRAT',
 'STRAT',
 'SNGLS',
 'SNGLS',
 'BQX',
 'BQX',
 'KNC',
 'KNC',
 'FUN',
 'FUN',
 'SNM',
 'SNM',
 'NEO',
 'IOTA',
 'IOTA',
 'LINK',
 'LINK',
 'XVG',
 'XVG',
 'SALT',
 'SALT',
 'MDA',
 'MDA',
 'MTL',
 'MTL',
 'SUB',
 'SUB',
 'EOS',
 'SNT',
 'ETC',
 'ETC',
 'MTH',
 'MTH',
 'ENG',
 'ENG',
 'DNT',
 'ZEC',
 'ZEC',
 'BNT',
 'AST',
 'AST',
 'DASH',
 'DASH',
 'OAX',
 'ICN',
 'BTG',
 'BTG',
 'EVX',
 'EVX',
 'REQ',
 'REQ',
 'VIB',
 'VIB',
 'HSR',
 'TRX',
 'TRX',
 'POWR',
 'POWR',
 'ARK',
 'ARK',
 'YOYO',
 'XRP',
 'XRP',
 'MOD',
 'MOD',
 'ENJ',
 'ENJ',
 'STORJ',
 'STORJ',
 'BNB',
 'VEN',
 'YOYO',
 'POWR',
 'VEN',
 'VEN',
 'KMD',
 'KMD',
 'NULS',
 'RCN',
 'RCN',
 'RCN',
 'NULS',
 'NULS',
 'RDN',
 'RDN',
 'RDN',
 'XMR',
 'XMR',
 'DLT',
 'WTC',
 'DLT',
 'DLT',
 

Get the order book for the required pair.

In [7]:
def orderBook(pair):
  r = requests.get(ENDPOINT + "api/v3/depth", params=dict(symbol=pair))
  return (json.loads(r.text))

In [8]:
orderBook('ETHBUSD')

{'lastUpdateId': 6939362873,
 'bids': [['4007.11000000', '0.28320000'],
  ['4007.00000000', '0.12480000'],
  ['4006.68000000', '0.24960000'],
  ['4006.46000000', '0.51040000'],
  ['4006.24000000', '1.00000000'],
  ['4006.02000000', '0.14560000'],
  ['4006.00000000', '1.09990000'],
  ['4005.69000000', '0.37000000'],
  ['4005.68000000', '1.73960000'],
  ['4005.62000000', '2.97000000'],
  ['4005.41000000', '1.11800000'],
  ['4005.08000000', '0.98240000'],
  ['4005.07000000', '2.00000000'],
  ['4005.01000000', '1.78940000'],
  ['4005.00000000', '0.12480000'],
  ['4004.95000000', '0.98690000'],
  ['4004.83000000', '0.18870000'],
  ['4004.79000000', '1.16950000'],
  ['4004.66000000', '2.00000000'],
  ['4004.44000000', '1.63630000'],
  ['4004.22000000', '0.24620000'],
  ['4004.19000000', '0.62390000'],
  ['4004.13000000', '2.00000000'],
  ['4004.00000000', '0.12490000'],
  ['4003.86000000', '1.67580000'],
  ['4003.85000000', '2.40900000'],
  ['4003.62000000', '1.66490000'],
  ['4003.43000000'

Get ask or bids for different cryptocurrencies.

In [9]:
def getDepth(direction, pair):
  r = orderBook(pair)
  return r[direction][0][0]

In [10]:
getDepth('asks','ETHBUSD')

'4007.40000000'

Get candles and indicators values.

In [11]:
def getCandleData(pair, duration):
  r = requests.get(ENDPOINT + "api/v3/klines", params=dict(symbol=pair, interval=duration))
  bars = json.loads(r.text)
  df = pd.DataFrame(bars, columns=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'No trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
  
  sma7 = SMAIndicator(df['Close'], 7, False)
  ema7 = EMAIndicator(df['Close'], 7, False)

  sma30 = SMAIndicator(df['Close'], 30, False)
  ema30 = EMAIndicator(df['Close'], 30, False)

  sma200 = SMAIndicator(df['Close'], 200, False)
  ema200 = EMAIndicator(df['Close'], 200, False)
  
  df['SMA 7'] = sma7.sma_indicator()
  df['EMA 7'] = ema7.ema_indicator()

  df['SMA 30'] = sma30.sma_indicator()
  df['EMA 30'] = ema30.ema_indicator()

  df['SMA 200'] = sma200.sma_indicator()
  df['EMA 200'] = ema200.ema_indicator()

  return (df)

In [12]:
getCandleData('ETHBUSD', '5m')

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,No trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore,SMA 7,EMA 7,SMA 30,EMA 30,SMA 200,EMA 200
0,1639459500000,3760.42000000,3774.00000000,3760.42000000,3773.30000000,518.45730000,1639459799999,1953259.37931700,1607,285.42910000,1075207.30810200,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1639459800000,3773.29000000,3786.00000000,3773.07000000,3783.72000000,626.37710000,1639460099999,2367282.42026700,2122,313.40290000,1184313.00257000,0,NaN,NaN,NaN,NaN,NaN,NaN
2,1639460100000,3783.72000000,3784.39000000,3772.66000000,3775.01000000,252.12960000,1639460399999,952647.86300800,810,144.03090000,544112.64251700,0,NaN,NaN,NaN,NaN,NaN,NaN
3,1639460400000,3775.21000000,3782.65000000,3771.77000000,3775.89000000,235.54670000,1639460699999,889919.28552400,844,151.13010000,571021.55218100,0,NaN,NaN,NaN,NaN,NaN,NaN
4,1639460700000,3775.18000000,3777.79000000,3764.20000000,3767.51000000,258.75580000,1639460999999,975101.92720600,943,118.28010000,445730.32516500,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1639608000000,4022.59000000,4029.99000000,4022.08000000,4022.09000000,218.96460000,1639608299999,881931.33966100,886,132.84900000,535140.03354600,0,4028.874286,4029.122122,4051.211000,4023.360924,3859.94385,3879.139388
496,1639608300000,4022.09000000,4031.81000000,4014.10000000,4014.10000000,394.74260000,1639608599999,1588408.15354900,1298,220.09000000,886016.38201100,0,4025.072857,4025.366591,4050.678333,4022.763445,3860.71795,3880.482280
497,1639608600000,4014.09000000,4014.99000000,4007.16000000,4008.91000000,280.80230000,1639608899999,1126430.77468700,1130,114.21520000,458155.35524900,0,4020.085714,4021.252443,4049.892667,4021.869674,3861.43765,3881.760168
498,1639608900000,4008.62000000,4008.92000000,3993.07000000,4001.34000000,2203.57260000,1639609199999,8814129.61810200,3730,1599.75870000,6398892.45281700,0,4015.480000,4016.274333,4048.010000,4020.545179,3862.09450,3882.950017


## Create and store to sql database

### Create tables

Create candles and indicators tables.

In [13]:
def createCandleTable(exchangeName, pair, duration):

    setTableName = str(exchangeName + "_" + pair + "_" + duration)
    conn = sqlite3.connect('candleDatabase.db')
    cur = conn.cursor()

    tableCreationStatement = """CREATE TABLE """ + setTableName + """(Id INTEGER PRIMARY KEY, date INT, high REAL, low REAL, open REAL, close REAL, volume REAL, quotevolume REAL, sma_7 REAL, ema_7 REAL, sma_30 REAL, ema_30 REAL, sma_200 REAL, ema_200 REAL)"""
    cur.execute(tableCreationStatement)

    conn.commit()
    conn.close()

In [14]:
createCandleTable('Binance', 'ETHBUSD', '5m')

Create table of dataset.

In [15]:
def createFullDatasetTable(exchangeName, pair):

    setTableName = str(exchangeName + "_" + pair)

    #return (setTableName)
    conn = sqlite3.connect('candleDatabase.db')
    cur = conn.cursor()

    tableCreationStatement = """CREATE TABLE """ + setTableName + """(Id INTEGER PRIMARY KEY, uuid TEXT, traded_btc REAL, price REAL, created_at_int INT, side TEXT)"""
    cur.execute(tableCreationStatement)

    conn.commit()
    conn.close()

In [16]:
createFullDatasetTable('Binance', 'BTCUSDT')

Create table to track last data.

In [17]:
def createTrackOfUpdateTable():

    conn = sqlite3.connect('candleDatabase.db')
    cur = conn.cursor()

    tableCreationStatement = """ CREATE TABLE last_checks(Id INTEGER PRIMARY KEY, exchange TEXT, trading_pair TEXT, duration TEXT, table_name TEXT, last_check INT, startdate INT, last_id INT) """
    cur.execute(tableCreationStatement)

    conn.commit()
    conn.close()

In [18]:
createTrackOfUpdateTable()

### Store to database

Store candles and indicators values to database.

In [19]:
def storeCandleData(pair, duration, database="candleDatabase.db"):

    setTableName = str("Binance_" + pair + "_" + duration)

    df = getCandleData(pair, duration)

    _date = df['Open time'].tolist()
    _open = df['Open'].tolist()
    _high = df['High'].tolist()
    _low = df['Low'].tolist()
    _close = df['Close'].tolist()
    _volume = df['Volume'].tolist()
    _quotevolume = df['Quote asset volume'].tolist()

    _sma7 = df['SMA 7'].tolist()
    _ema7 = df['EMA 7'].tolist()
    _sma30 = df['SMA 30'].tolist()
    _ema30 = df['EMA 30'].tolist()
    _sma200 = df['SMA 200'].tolist()
    _ema200 = df['EMA 200'].tolist()

    conn = sqlite3.connect(database, timeout=20)
    cur = conn.cursor()
    for tx in range(len(df)):
        id = _date[tx]
        date = _date[tx]
        open = _open[tx]
        high = _high[tx]
        low = _low[tx]
        close = _close[tx]
        volume = _volume[tx]
        quotevolume = _quotevolume[tx]

        sma7 = _sma7[tx]
        ema7 = _ema7[tx]
        sma30 = _sma30[tx]
        ema30 = _ema30[tx]
        sma200 = _sma200[tx]
        ema200 = _ema200[tx]

        data = (id, date, open, high, low, close, volume, quotevolume, sma7, ema7, sma30, ema30, sma200, ema200)
        cur.executemany("INSERT INTO " + setTableName + " (id, date, open, high, low, close, volume, quotevolume, sma_7, ema_7, sma_30, ema_30, sma_200, ema_200) VALUES ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", (data,))
        # the secure way to enter the variable
    conn.commit()
    conn.close() 

In [20]:
storeCandleData('ETHBUSD', '5m')

# POST orders

Set values.

In [36]:
API_KEY = ''
API_SECRET = ''
order_api_url = 'api/v3/order'

In [37]:
def get_signed_url(dataQueryString, secret_key):
    signature = hmac.new(API_SECRET.encode(), dataQueryString.encode(), 'sha256').hexdigest()
    return (ENDPOINT + order_api_url + '?' + dataQueryString + '&signature=' + signature)

POST a limit order.

In [41]:
def createOrder(api_key, secret_key, direction, price, amount, pair='BTCUSDT', orderType='LimitOrder'):
    type = ''
    if orderType == 'LimitOrder':
        type = 'LIMIT'
    
    # dataQueryString = 'symbol=BTCUSDT&side=BUY&type=LIMIT&timeInForce=GTC&quantity=0.0015&price=30000&recvWindow=20000&timestamp=' + str(int(time.time()*1000))
    dataQueryString = 'symbol=' + pair +'&side=' + direction.upper() + '&type=' + type + '&timeInForce=GTC&quantity=' + str(amount) + '&price=' + str(price) + '&recvWindow=20000&timestamp=' + str(int(time.time()*1000))
    
    headers = {'X-MBX-APIKEY': api_key}
    url = get_signed_url(dataQueryString, secret_key)

    response = requests.post(url, headers=headers)
    r = json.loads(response.text)

    print(r['orderId'])
    return r

In [62]:
# # Call create order and save to database

# exchangeName = 'Binance'
# pair = 'BTCUSDT'
# setTableName = str(exchangeName + "_" + pair)
# database="candleDatabase.db"

# r = createOrder(API_KEY, API_SECRET, 'buy', 30000, 0.001, pair)
# print(r)

# # Save to sqlite
# conn = sqlite3.connect(database, timeout=20)
# cur = conn.cursor()

# id = r['orderId']
# uuid = str(r['orderId'])
# traded_btc = float(r['origQty'])
# price = float(r['price'])
# created_at_int = r['transactTime']
# side = r['side']

# data = (id, uuid, traded_btc, price, created_at_int, side)
# cur.executemany("INSERT INTO " + setTableName + " (id, uuid, traded_btc, price, created_at_int, side) VALUES ( ?, ?, ?, ?, ?, ?)", (data,))
# # the secure way to enter the variable
# conn.commit()
# conn.close()

8645144292
{'symbol': 'BTCUSDT', 'orderId': 8645144292, 'orderListId': -1, 'clientOrderId': 'xJCP6NevOgBdycXaL85bIj', 'transactTime': 1639610929548, 'price': '30000.00000000', 'origQty': '0.00100000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'NEW', 'timeInForce': 'GTC', 'type': 'LIMIT', 'side': 'BUY', 'fills': []}


DELETE a limit order.

In [43]:
def cancelOrder(api_key, secret_key, uuid, pair='BTCUSDT'):
    
    # dataQueryString = 'symbol=BTCUSDT&orderId=8617530103&recvWindow=20000&timestamp=' + str(int(time.time()*1000))
    dataQueryString = 'symbol=' + pair + '&orderId=' + str(uuid) + '&recvWindow=20000&timestamp=' + str(int(time.time()*1000))
    
    headers = {'X-MBX-APIKEY': api_key}
    url = get_signed_url(dataQueryString, secret_key)

    response = requests.delete(url, headers=headers)
    r = json.loads(response.text)

    return r

In [63]:
# # Call delete order and remove from database

# exchangeName = 'Binance'
# pair = 'BTCUSDT'
# setTableName = str(exchangeName + "_" + pair)
# database="candleDatabase.db"
# uuid = 8645144292

# r = cancelOrder(API_KEY, API_SECRET, uuid)
# print(r)

# # Remove from sqlite
# conn = sqlite3.connect(database, timeout=20)
# cur = conn.cursor()

# cur.execute("DELETE FROM " + setTableName + " WHERE Id = " + str(uuid))
# # the secure way to enter the variable
# conn.commit()
# conn.close()

{'symbol': 'BTCUSDT', 'origClientOrderId': 'xJCP6NevOgBdycXaL85bIj', 'orderId': 8645144292, 'orderListId': -1, 'clientOrderId': 'SSZT9VrbHmxEvLpjAxPgJy', 'price': '30000.00000000', 'origQty': '0.00100000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'CANCELED', 'timeInForce': 'GTC', 'type': 'LIMIT', 'side': 'BUY'}
